<a href="https://colab.research.google.com/github/Elwing-Chou/ml0716/blob/master/news.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import glob
import os
import pandas as pd

def getdf(targetdn):
    datas = {
        "path":[],
        "label":[]
    }
    base = "drive/My Drive/0724test"
    p = os.path.join(base, targetdn, "*")
    for dn in glob.glob(p):
        name = os.path.split(dn)[-1]
        fn = os.path.join(dn, "*.[tT][xX][tT]")
        fns = glob.glob(fn)
        datas["label"] = datas["label"] + [name] * len(fns)
        datas["path"] = datas["path"] + fns
    return pd.DataFrame(datas)

In [ ]:
train_df = getdf("chinese_news_trans")
test_df = getdf("chinese_news_test")

In [ ]:
def read_content(fn):
    with open(fn, "r", encoding="utf-8") as f:
        content = f.read()
    return content
train_df["content"] = train_df["path"].apply(read_content)
test_df["content"] = test_df["path"].apply(read_content)